In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import numpy as np
import io
import random
import math
import scipy.stats
import matplotlib.pylab as plt
import seaborn as sns

C:\Users\devin\AppData\Local\Temp\ipykernel_23044\4169296045.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# Initialize year variable
year = 2025 #(2019, 2021, 2022, 2023, 2024, 2025)

In [3]:
# Read in data based on the year
if year == 2019:
    data = pd.read_csv('data/18_19tourneycs.csv')
elif year == 2021:
    data = pd.read_csv('data/20_21tourneycs.csv')
elif year == 2022:
    data = pd.read_csv('data/21_22tourneycs.csv')
elif year == 2023:
    data = pd.read_csv('data/22_23tourneycs.csv')
elif year == 2024:
    data = pd.read_csv('data/23_24tourneycs.csv')
elif year == 2025:
    data = pd.read_csv('data/24_25tourneycs.csv')
data.head()

,Region,Seed,School,SRS
0,1,1,Auburn,29.58
1,1,16,Alabama State,-9.22
2,1,8,Louisville,19.30
3,1,9,Creighton,16.36
4,1,4,Texas A&M,20.05


In [4]:
# Rename columns of data DataFrame
data.columns = ['Region', 'Seed', 'Team', 'SRS']
data

,Region,Seed,Team,SRS
0,1,1,Auburn,29.58
1,1,16,Alabama State,-9.22
2,1,8,Louisville,19.30
3,1,9,Creighton,16.36
4,1,4,Texas A&M,20.05
...,...,...,...,...
59,4,11,Xavier,15.57
60,4,7,UCLA,19.24
61,4,10,Utah State,14.48
62,4,2,Tennessee,25.04


In [5]:
num_of_sims = 1000  # Number of simulations
teams = data.to_numpy().tolist()  # Convert DataFrame to a nested list
team_dict = {}  # Dictionary to store team data

# Iterate through each team
for i in range(0, len(teams)):
    # Initialize an empty list for each team in the team_dict
    team_dict[teams[i][2]] = [0, 0, 0, 0, 0, 0, 0]
    # The team name is accessed using teams[i][2], assuming the team name is in the third column of the data

In [6]:
"""
Given a list of teams in a specific round and the round number,
find_winners randomly determines the winners of each matchup based on the
SRS difference between the teams and returns a list of the winners
"""
def find_winners(input_list, round):

    winners = []

    # Iterate over the input list of teams
    for i in range(0, len(input_list)):
        # Skips if the index is odd (represents the second team in a matchup)
        if i % 2 != 0:
            pass
        else:
            # Get the first team and the second team in the matchup
            team1 = input_list[i]
            team2 = input_list[i+1]

            # Calculate the SRS difference between the teams
            srs_diff = team1[3] - team2[3]

            # Calculate the win probability of Team 1 using the SRS difference
            team1_prob = scipy.stats.norm(0, 10.36).cdf(srs_diff)

            # Generate a random draw between 0 and 1
            draw = random.random()

            # Determine the winner based on the draw and win probability
            if draw <= team1_prob:
                # Team 1 wins
                winners.append(team1)
                team_dict[team1[2]][round] += 1
            else:
                # Team 2 wins
                winners.append(team2)
                team_dict[team2[2]][round] += 1

    return winners


In [7]:
"""
Prints out the bracket representation of the tournament rounds
based on the provided round_list
"""
def bracket_printout(round_list):

    for i in range(0, len(round_list)):
        temp_list = round_list[i]

        # Iterate over the matchups in the current round
        for j in range(0, len(temp_list)):
            # Print the team seed, name, and SRS for each matchup
            print(str(temp_list[j][1]) + " " + temp_list[j][2] + " " + str(temp_list[j][3]))

            # Print an empty line after every even-indexed matchup
            if j % 2 != 0:
                print("")

        # Print empty lines between each round
        print("")
        print("")
        print("")


In [8]:
"""
Simulates a tournament by finding winners in each round and printing the bracket
"""
def tourney_sim(teams):
    # Increment the count of tournaments for each team
    for x in team_dict:
        team_dict[x][0] += 1

    # Find winners in the Round of 64
    round_32 = find_winners(teams, 1)

    # Find winners in the Round of 32
    sweet_16 = find_winners(round_32, 2)

    # Find winners in the Sweet 16
    elite_8 = find_winners(sweet_16, 3)

    # Find winners in the Elite 8
    final_4 = find_winners(elite_8, 4)

    # Find winners in the Final 4
    championship = find_winners(final_4, 5)

    # Find the winner in the Championship
    winner = find_winners(championship, 6)

    # Print the bracket for all rounds
    # Uncomment below to see bracket format of simulation. Heads up: it is a lot of output!
    #bracket_printout([teams, round_32, sweet_16, elite_8, final_4, championship, winner])


In [9]:
%%time
# Run the monte-carlo tournament simulation
for i in range(1, num_of_sims+1):
    tourney_sim(teams)

# Calculate the average statistics for each team
for x in team_dict:
    # Divide the count of wins in each round by the total number of simulations
    for i in range(0, len(team_dict[x])):
        team_dict[x][i] = team_dict[x][i] / num_of_sims

    # Print the team name and its average statistics
    print(x + " " + str(team_dict[x]))


Auburn [1.0, 1.0, 0.88, 0.738, 0.568, 0.386, 0.225]
Alabama State [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Louisville [1.0, 0.593, 0.085, 0.041, 0.026, 0.006, 0.002]
Creighton [1.0, 0.407, 0.035, 0.011, 0.002, 0.0, 0.0]
Texas A&M [1.0, 0.907, 0.507, 0.111, 0.051, 0.015, 0.002]
Yale [1.0, 0.093, 0.011, 0.0, 0.0, 0.0, 0.0]
Michigan [1.0, 0.728, 0.403, 0.092, 0.041, 0.016, 0.005]
UC San Diego [1.0, 0.272, 0.079, 0.007, 0.0, 0.0, 0.0]
Iowa State [1.0, 0.965, 0.666, 0.376, 0.145, 0.057, 0.014]
Lipscomb [1.0, 0.035, 0.002, 0.0, 0.0, 0.0, 0.0]
Mississippi [1.0, 0.548, 0.177, 0.061, 0.008, 0.002, 0.0]
North Carolina [1.0, 0.452, 0.155, 0.052, 0.011, 0.004, 0.0]
Marquette [1.0, 0.664, 0.221, 0.102, 0.013, 0.004, 0.001]
New Mexico [1.0, 0.336, 0.076, 0.021, 0.005, 0.001, 0.001]
Michigan State [1.0, 0.991, 0.703, 0.388, 0.13, 0.059, 0.018]
Bryant [1.0, 0.009, 0.0, 0.0, 0.0, 0.0, 0.0]
Florida [1.0, 0.998, 0.847, 0.605, 0.417, 0.233, 0.118]
Norfolk State [1.0, 0.002, 0.0, 0.0, 0.0, 0.0, 0.0]
Connecticut

In [10]:
point_system = [0, 1, 2, 3, 5, 8, 13]  # Predefined point system for each round

expected_values = {}  # Dictionary to store the expected value for each team

# Calculate the expected value for each team
for team, probabilities in team_dict.items():
    expected_value = sum(probabilities[i] * point_system[i] for i in range(len(probabilities)))
    expected_values[team] = expected_value

# Sort teams by expected value in descending order
sorted_teams = sorted(expected_values.items(), key=lambda x: x[1], reverse=True)

# Print the expected value for each team
for team, expected_value in sorted_teams:
    print(f"{team}: {expected_value:.3f}")

Auburn: 13.827
Duke: 12.570
Florida: 9.990
Houston: 9.462
Alabama: 7.926
Tennessee: 6.727
Maryland: 5.324
Texas Tech: 5.130
Michigan State: 4.917
Iowa State: 4.788
Arizona: 4.265
St. John's: 4.219
Wisconsin: 3.540
Kentucky: 3.310
Purdue: 3.241
Illinois: 2.710
Texas A&M: 2.655
Missouri: 2.522
Gonzaga: 2.402
Michigan: 2.208
Clemson: 2.056
Kansas: 1.878
BYU: 1.746
Oregon: 1.554
Marquette: 1.522
UCLA: 1.441
Mississippi: 1.141
Saint Mary's: 1.133
Louisville: 1.090
North Carolina: 1.005
Connecticut: 0.993
Baylor: 0.921
Colorado State: 0.882
Arkansas: 0.702
Mississippi State: 0.659
Memphis: 0.625
VCU: 0.623
Oklahoma: 0.613
New Mexico: 0.597
Xavier: 0.546
Vanderbilt: 0.525
Creighton: 0.520
Utah State: 0.462
UC San Diego: 0.451
Georgia: 0.437
McNeese State: 0.258
Liberty: 0.233
Drake: 0.168
Yale: 0.115
Grand Canyon: 0.089
High Point: 0.080
Troy: 0.048
Akron: 0.045
Lipscomb: 0.039
UNC Wilmington: 0.022
Omaha: 0.015
Montana: 0.013
Bryant: 0.009
Robert Morris: 0.005
Wofford: 0.003
Norfolk State: 0

In [11]:
# Convert dictionary to DataFrame
df_expected_values = pd.DataFrame.from_dict(expected_values, orient='index', columns=['Expected Value'])

# Sort teams by expected value in descending order
df_expected_values = df_expected_values.sort_values(by='Expected Value', ascending=False)

# Reset the index
df_expected_values = df_expected_values.reset_index().rename(columns={'index': 'Team'})

# Display the DataFrame
display(df_expected_values)

,Team,Expected Value
0,Auburn,13.827
1,Duke,12.570
2,Florida,9.990
3,Houston,9.462
4,Alabama,7.926
...,...,...
59,Wofford,0.003
60,Norfolk State,0.002
61,SIU Edwardsville,0.001
62,Alabama State,0.000


In [12]:
# Merge the dataframes based on the 'Team' column
df_merged = data.merge(df_expected_values, on='Team')

# Display the merged DataFrame
df_merged.head()

,Region,Seed,Team,SRS,Expected Value
0,1,1,Auburn,29.58,13.827
1,1,16,Alabama State,-9.22,0.000
2,1,8,Louisville,19.30,1.090
3,1,9,Creighton,16.36,0.520
4,1,4,Texas A&M,20.05,2.655


In [13]:
# Define the mapping of seed values to cost values
seed_cost_mapping = {
    1: 75,
    2: 40,
    3: 25,
    4: 20,
    5: 17,
    6: 15,
    7: 12,
    8: 10,
    9: 9,
    10: 8,
    11: 7,
    12: 6,
    13: 5,
    14: 4,
    15: 3,
    16: 1,
}

# Add a new column 'Cost' to df_merged based on the 'Seed' column
df_merged['Cost'] = df_merged['Seed'].map(seed_cost_mapping)

# Display the merged DataFrame
df_merged.head()

,Region,Seed,Team,SRS,Expected Value,Cost
0,1,1,Auburn,29.58,13.827,75
1,1,16,Alabama State,-9.22,0.000,1
2,1,8,Louisville,19.30,1.090,10
3,1,9,Creighton,16.36,0.520,9
4,1,4,Texas A&M,20.05,2.655,20


In [14]:
# Define the knapsack function
def knapsack(items, budget):
    n = len(items)
    dp = [[0] * (budget + 1) for _ in range(n + 1)]

    for i in range(1, n + 1):
        for j in range(1, budget + 1):
            cost = items[i - 1]['Cost']
            value = items[i - 1]['Expected Value']
            if cost > j:
                dp[i][j] = dp[i - 1][j]
            else:
                dp[i][j] = max(dp[i - 1][j], value + dp[i - 1][j - cost])

    selected = []
    j = budget
    for i in range(n, 0, -1):
        if dp[i][j] != dp[i - 1][j]:
            selected.append(items[i - 1])
            j -= items[i - 1]['Cost']

    return selected

# Run the knapsack optimization on df_merged
selected_teams = knapsack(df_merged.to_dict('records'), budget=224)

# Create a new DataFrame for the selected teams
df_selected = pd.DataFrame(selected_teams)

# Display the selected teams and their information
display(df_selected)

# Calculate the total sum of the cost and expected value columns
total_cost = df_selected['Cost'].sum()
total_expected_value = df_selected['Expected Value'].sum()

# Print the total sums
print(f"Total Cost: {total_cost}")
print(f"Total Expected Value: {total_expected_value:.3f}")

,Region,Seed,Team,SRS,Expected Value,Cost
0,4,8,Gonzaga,23.63,2.402,10
1,4,16,SIU Edwardsville,-5.83,0.001,1
2,3,2,Alabama,25.97,7.926,40
3,3,4,Arizona,23.14,4.265,20
4,2,3,Texas Tech,23.91,5.130,25
5,2,4,Maryland,23.54,5.324,20
6,2,16,Norfolk State,-3.61,0.002,1
7,1,11,North Carolina,17.17,1.005,7
8,1,3,Iowa State,23.05,4.788,25
9,1,1,Auburn,29.58,13.827,75


Total Cost: 224
Total Expected Value: 44.670
